In [ ]:
# 攻撃後多次元配列(.pt)へ変換
import os
import re
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import torchvision.transforms as T
from torchvision import models
from torch.nn import functional as F
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import foolbox as fb

# --- デバイス ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 画像前処理（入力は [0,1]）---
transform_base = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()  # -> [0,1]
])

# --- 画像前処理（抽出スクリプトと統一）---
transform_base = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

# --- モデル準備（NormalizeModule不要）---
mobilenetv2 = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
mobilenetv2 = mobilenetv2.to(device).eval()

# Foolbox用モデル（正規化後なので bounds は -3〜3）
fmodel = fb.PyTorchModel(mobilenetv2, bounds=(-3, 3))

imagenet_classes = models.MobileNet_V2_Weights.IMAGENET1K_V1.meta["categories"]


# --- ヘルパー関数 ---
def decode_img(img_path):
    img = Image.open(img_path).convert("RGB")
    return transform_base(img).unsqueeze(0)

def to_torch_clipped(x):
    if isinstance(x, np.ndarray):
        t = torch.from_numpy(x)
    elif isinstance(x, torch.Tensor):
        t = x.clone().detach()
    else:
        raise TypeError("Unsupported type for clipped output")
    if t.dim() == 3:
        if t.shape[2] in (1,3) and t.shape[0] not in (1,3):
            t = t.permute(2,0,1).unsqueeze(0)
        else:
            t = t.unsqueeze(0)
    return t.float().cpu().clamp(0,1)

def save_tensor_image_0_1(tensor, path):
    t = tensor.squeeze(0).cpu().clamp(0,1)
    arr = (t.numpy() * 255).astype('uint8')
    arr = np.transpose(arr, (1,2,0))
    Image.fromarray(arr).save(path)

def save_noise_vis(orig, clipped, path):
    o = orig.squeeze(0).cpu().numpy()
    c = clipped.squeeze(0).cpu().numpy()
    diff = o - c
    mag = np.max(np.abs(diff))
    vis = (diff / (2*mag) + 0.5) if mag != 0 else np.zeros_like(diff)
    arr = (np.transpose(vis, (1,2,0)) * 255).astype('uint8')
    Image.fromarray(arr).save(path)

def extract_number(path):
    match = re.search(r'\d+', path.stem)
    return int(match.group()) if match else float('inf')

# --- モデル準備 ---
mobilenetv2 = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
mobilenetv2.eval()


imagenet_classes = models.MobileNet_V2_Weights.DEFAULT.meta["categories"]

# --- 推論関数 ---
def pred(images, topk=1):
    if isinstance(images, np.ndarray):
        images = torch.from_numpy(images)
    images = images.to(device).float()
    with torch.no_grad():
        logits = model_for_foolbox(images)
        probs = F.softmax(logits, dim=1)
    top_probs, top_idxs = torch.topk(probs, topk)
    return [(idx.item(), idx.item(), p.item()) for idx, p in zip(top_idxs[0], top_probs[0])]

def lb_index(prediction):
    if isinstance(prediction, int):
        return prediction
    elif isinstance(prediction, (tuple, list)):
        return prediction[0]
    else:
        return prediction

def lb_to_tensor(idx):
    return torch.tensor([idx], dtype=torch.long, device=device)

# --- FGSM 攻撃 ---
def perform_attack(fmodel, images, labels, steps=1000, max_eps=1.0):
    attack = fb.attacks.FGSM()
    eps = 0.0
    for n in range(steps):
        eps += max_eps / steps
        raw, clipped, is_adv = attack(fmodel, images, labels, epsilons=[eps])
        if isinstance(clipped, (list, tuple)):
            clipped = clipped[0]
        clipped_t = to_torch_clipped(clipped)
        adv_lb = lb_index(pred(clipped_t.to(device), 1)[0][1])
        orig_lb = lb_index(pred(images, 1)[0][1])
        if adv_lb != orig_lb:
            return clipped_t, eps, adv_lb
    return None, eps, lb_index(pred(images, 1)[0][1])

# --- ディレクトリ設定 ---
folder_path = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\data1")
output_adv_dir = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\data1_FGSM")
output_noise_dir = output_adv_dir / "noise"
output_adv_tensor_dir = output_adv_dir / "tensor"

output_adv_dir.mkdir(parents=True, exist_ok=True)
output_noise_dir.mkdir(parents=True, exist_ok=True)
output_adv_tensor_dir.mkdir(parents=True, exist_ok=True)

# --- 攻撃パラメータ ---
steps = 1000
max_eps = 1.0

# --- 攻撃実行 ---
AE_results = []
image_files = sorted([f for f in folder_path.iterdir() if f.suffix.lower() in ['.jpg','.jpeg','.png']], key=extract_number)

for i, img_path in enumerate(image_files):
    images = decode_img(str(img_path)).to(device)
    labels = lb_to_tensor(lb_index(pred(images,1)[0][1]))
    clipped, eps, adv_lb = perform_attack(fmodel, images, labels, steps=steps, max_eps=max_eps)
    pred_orig = pred(images,1)
    orig_label, orig_conf = pred_orig[0][1], pred_orig[0][2]

    if clipped is not None:
        pred_adv = pred(clipped.to(device),1)
        adv_label, adv_conf = pred_adv[0][1], pred_adv[0][2]

        adv_tensor_path = output_adv_tensor_dir / f"{img_path.stem}.pt"
        torch.save(to_torch_clipped(clipped), adv_tensor_path)

        adv_path = output_adv_dir / f"{img_path.stem}.jpg"
        save_tensor_image_0_1(clipped, adv_path)
        save_noise_vis(images.cpu(), clipped, output_noise_dir / f"noise{i}.jpg")

        AE_results.append({
            'image': img_path.name,
            'Original_label_id': orig_label,
            'Original_label_name': imagenet_classes[orig_label],
            'Original_confidence': orig_conf,
            'Advs_label_id': adv_label,
            'Advs_label_name': imagenet_classes[adv_label],
            'Advs_confidence': adv_conf,
            'Advs_eps': eps,
            'Advs_tensor_path': str(adv_tensor_path)
        })

        print(f"[{i}] 攻撃成功: Eps={eps:.4f}, {imagenet_classes[orig_label]} → {imagenet_classes[adv_label]}")
    else:
        print(f"[{i}] 攻撃失敗: {imagenet_classes[orig_label]}")

# --- CSV保存 ---
csv_path = output_adv_dir / "Origin_Adv.csv"
pd.DataFrame(AE_results).to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"\n✅ 攻撃結果を保存しました: {csv_path}")


[0] 攻撃成功: Eps=0.0010, space shuttle → drilling platform
[1] 攻撃成功: Eps=0.0010, drake → red-breasted merganser
[2] 攻撃成功: Eps=0.1220, dishrag → jellyfish
[3] 攻撃成功: Eps=0.1850, green snake → nematode
[4] 攻撃成功: Eps=0.0010, miniskirt → jean
[5] 攻撃成功: Eps=0.0010, stupa → chime
[6] 攻撃成功: Eps=0.0040, toucan → king penguin
[7] 攻撃成功: Eps=0.0220, limpkin → bittern
[8] 攻撃成功: Eps=0.0020, chickadee → thimble
[9] 攻撃成功: Eps=0.0010, barbell → tricycle
[10] 攻撃成功: Eps=0.0010, bottlecap → orange
[11] 攻撃成功: Eps=0.0010, submarine → water tower
[12] 攻撃成功: Eps=0.0010, fur coat → trench coat
[13] 攻撃成功: Eps=0.0010, cicada → bittern
[14] 攻撃成功: Eps=0.0010, spindle → clog
[15] 攻撃成功: Eps=0.0010, Christmas stocking → umbrella
[16] 攻撃成功: Eps=0.0770, electric locomotive → garbage truck
[17] 攻撃成功: Eps=0.0640, guenon → shower curtain
[18] 攻撃成功: Eps=0.0010, go-kart → lawn mower
[19] 攻撃成功: Eps=0.0050, Angora → Samoyed
[20] 攻撃成功: Eps=0.0030, half track → tow truck
[21] 攻撃成功: Eps=0.0020, trench coat → bulletproof vest
[22] 攻

In [3]:
# FGSM attack + correct save/unnormalize workflow (修正版：bounds scalar化済み)
import os
import re
import numpy as np
import torch
import torchvision.transforms as T
from torchvision import models
import torch.nn.functional as F
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import foolbox as fb
import pandas as pd

# === 設定（環境に合わせて変更）===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
folder_path = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\data1")  # 入力画像フォルダ（抽出済み）
output_adv_dir = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\data1_FGSM")
output_noise_dir = output_adv_dir / "noise"
output_adv_tensor_dir = output_adv_dir / "tensor"
for p in [output_adv_dir, output_noise_dir, output_adv_tensor_dir]:
    p.mkdir(parents=True, exist_ok=True)

# --- 前処理（抽出スクリプトと完全一致）---
transform_base = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

# --- ImageNet 逆正規化用定数 ---
MEAN = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float32)
STD  = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32)

# ----------------------
# --- ヘルパー関数（差し替え済み）---
# ----------------------
def decode_img(img_path):
    """画像を読み込みテンソルに変換（正規化済み） -> shape (1,3,H,W)"""
    img = Image.open(img_path).convert("RGB")
    return transform_base(img).unsqueeze(0)

def to_torch_clipped(x):
    """Foolbox出力（numpy or torch, 形状いろいろ）を (1,3,H,W) の torch.Tensor に変換して CPU に置く"""
    if isinstance(x, np.ndarray):
        t = torch.from_numpy(x)
    elif isinstance(x, torch.Tensor):
        t = x.clone().detach()
    else:
        raise TypeError("Unsupported type for clipped output")
    # normalize dims to (1,3,H,W)
    if t.dim() == 4:
        pass  # assume (B,C,H,W)
    elif t.dim() == 3:
        # could be (C,H,W) or (H,W,C)
        if t.shape[0] in (1,3):
            t = t.unsqueeze(0)  # (1,C,H,W)
        else:
            t = t.permute(2,0,1).unsqueeze(0)  # (H,W,C) -> (1,C,H,W)
    elif t.dim() == 2:
        t = t.unsqueeze(0).unsqueeze(0)  # (H,W) -> (1,1,H,W)
    return t.float().cpu()

def unnormalize_tensor(t):
    """正規化済みテンソルを [0,1] に戻す
       入力: (1,3,H,W) または (3,H,W)
       出力: (3,H,W) tensor in [0,1]
    """
    if t.dim() == 4:
        t = t.squeeze(0)
    t = t.cpu()
    t = t * STD.view(3,1,1) + MEAN.view(3,1,1)
    t = torch.clamp(t, 0.0, 1.0)
    return t

def save_tensor_image_0_1(tensor, path):
    """逆正規化して [0,1] -> uint8 にして保存"""
    t = unnormalize_tensor(t)  # (3,H,W)
    arr = (t.numpy() * 255).astype('uint8')
    arr = np.transpose(arr, (1,2,0))
    Image.fromarray(arr).save(path)

def save_noise_vis(orig, clipped, path):
    """orig, clipped は (1,3,H,W) 正規化済みテンソル（CPUでも可）"""
    o = orig.squeeze(0).cpu().numpy()
    c = clipped.squeeze(0).cpu().numpy()
    diff = o - c
    mag = np.max(np.abs(diff))
    eps_vis = 1e-6
    vis = (diff / (2 * (mag + eps_vis)) + 0.5)
    arr = (np.transpose(vis, (1, 2, 0)) * 255).astype('uint8')
    Image.fromarray(arr).save(path)

def extract_number(path: Path):
    """ファイル名の最初の数字を取り出す（ソート用）"""
    match = re.search(r'\d+', path.stem)
    return int(match.group()) if match else float('inf')

# ----------------------
# --- モデル準備（抽出コードと統一）---
# ----------------------
mobilenetv2 = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
mobilenetv2 = mobilenetv2.to(device).eval()

# --- Foolbox model bounds: torchvision Normalize を適用した後の入力範囲を scalar に縮約して指定 ---
per_channel_lower = (0.0 - np.array([0.485,0.456,0.406])) / np.array([0.229,0.224,0.225])
per_channel_upper = (1.0 - np.array([0.485,0.456,0.406])) / np.array([0.229,0.224,0.225])
lower = float(per_channel_lower.min())  # scalar 下限 (例: ~ -2.1179)
upper = float(per_channel_upper.max())  # scalar 上限 (例: ~  2.6400)
fmodel = fb.PyTorchModel(mobilenetv2, bounds=(lower, upper))

imagenet_classes = models.MobileNet_V2_Weights.IMAGENET1K_V1.meta["categories"]

# --- 推論関数 ---
def pred(images, topk=1):
    """images: torch.Tensor (B,C,H,W) or numpy; returns list of (idx, name, prob) for topk"""
    if isinstance(images, np.ndarray):
        images = torch.from_numpy(images)
    images = images.to(device).float()
    with torch.no_grad():
        logits = mobilenetv2(images)
        probs = F.softmax(logits, dim=1)
    top_probs, top_idxs = torch.topk(probs, topk)
    return [(int(idx.item()), imagenet_classes[int(idx.item())], float(p.item()))
            for idx, p in zip(top_idxs[0], top_probs[0])]

def lb_to_tensor(idx):
    return torch.tensor([idx], dtype=torch.long, device=device)

# ----------------------
# --- FGSM 攻撃部（そのまま使える）---
# ----------------------
attack = fb.attacks.FGSM()

def perform_attack(fmodel, images, labels, steps=1000, max_eps=1.0):
    """eps を増やして最小の破壊 eps を見つける探索"""
    eps = 0.0
    for n in range(steps):
        eps += max_eps / steps
        raw, clipped, is_adv = attack(fmodel, images, labels, epsilons=[eps])
        if isinstance(clipped, (list, tuple)):
            clipped = clipped[0]
        clipped_t = to_torch_clipped(clipped)
        adv_lb = pred(clipped_t.to(device), 1)[0][0]
        orig_lb = pred(images, 1)[0][0]
        if adv_lb != orig_lb:
            return clipped_t, eps, adv_lb
    return None, eps, pred(images, 1)[0][0]

# ----------------------
# --- 実行ループ ---
# ----------------------
steps = 1000
max_eps = 1.0

image_files = sorted(
    [f for f in folder_path.iterdir() if f.suffix.lower() in ['.jpg', '.jpeg', '.png']],
    key=extract_number
)

AE_results = []
for i, img_path in enumerate(tqdm(image_files, desc="FGSM Attack")):
    images = decode_img(str(img_path)).to(device)  # (1,3,H,W) normalized
    orig_pred = pred(images, 1)
    orig_label, orig_name, orig_conf = orig_pred[0]

    labels = lb_to_tensor(orig_label)
    clipped, eps, adv_lb = perform_attack(fmodel, images, labels, steps=steps, max_eps=max_eps)

    if clipped is not None:
        adv_pred = pred(clipped.to(device), 1)
        adv_label, adv_name, adv_conf = adv_pred[0]

        adv_tensor_path = output_adv_tensor_dir / f"{img_path.stem}.pt"
        # saved tensor is normalized (same scale as model expects) — good for re-loading later
        torch.save(to_torch_clipped(clipped), adv_tensor_path)

        adv_path = output_adv_dir / f"{img_path.stem}.jpg"
        save_tensor_image_0_1(clipped, adv_path)  # 逆正規化して保存

        save_noise_vis(images.cpu(), clipped, output_noise_dir / f"noise_{i}.jpg")

        AE_results.append({
            'image': img_path.name,
            'Original_label_id': orig_label,
            'Original_label_name': orig_name,
            'Original_confidence': orig_conf,
            'Advs_label_id': adv_label,
            'Advs_label_name': adv_name,
            'Advs_confidence': adv_conf,
            'Advs_eps': eps,
            'Advs_tensor_path': str(adv_tensor_path)
        })

        print(f"[{i}] 攻撃成功: Eps={eps:.4f}, {orig_name} → {adv_name}")
    else:
        print(f"[{i}] 攻撃失敗: {orig_name}")

# --- CSV保存 ---
csv_path = output_adv_dir / "Origin_Adv.csv"
pd.DataFrame(AE_results).to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"\n✅ 攻撃結果を保存しました: {csv_path}")


FGSM Attack:   0%|          | 0/1000 [00:00<?, ?it/s]


AssertionError: 

In [2]:
# ==========================================================
# fgsm_attack_fixed_v4.py
# FGSM攻撃（抽出スクリプト完全一致 + torchvision全バージョン対応）
# モデル: MobileNetV2(IMAGENET1K_V1)
# mean/std: meta or transforms() 自動判定
# Foolbox bounds=(0,1)
# ==========================================================

import os, re, torch, numpy as np, pandas as pd
import torchvision.transforms as T
from torchvision import models
from torch.nn import functional as F
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import foolbox as fb
import torch.nn as nn

# --- デバイス ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- パス設定 ---
folder_path = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\data3")
output_adv_dir = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\data3_FGSM")
output_noise_dir = output_adv_dir / "noise"
output_adv_tensor_dir = output_adv_dir / "tensor"
for p in [output_adv_dir, output_noise_dir, output_adv_tensor_dir]:
    p.mkdir(parents=True, exist_ok=True)

# --- 前処理（Normalizeしない）---
transform_base = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()   # -> [0,1]
])

# --- MobileNetV2 (IMAGENET1K_V1) ---
weights = models.MobileNet_V2_Weights.IMAGENET1K_V1
mobilenetv2 = models.mobilenet_v2(weights=weights).to(device).eval()

# torchvision のバージョン差を吸収
if "mean" in weights.meta and "std" in weights.meta:
    mean, std = weights.meta["mean"], weights.meta["std"]
else:
    tinfo = weights.transforms()
    mean, std = tinfo.mean, tinfo.std

imagenet_classes = weights.meta.get("categories", None)
if imagenet_classes is None:
    imagenet_classes = [f"class_{i}" for i in range(1000)]

# --- Normalize層 ---
class NormalizeModule(nn.Module):
    def __init__(self, mean, std):
        super().__init__()
        self.register_buffer("mean", torch.tensor(mean).view(1, 3, 1, 1))
        self.register_buffer("std", torch.tensor(std).view(1, 3, 1, 1))
    def forward(self, x):
        return (x - self.mean) / self.std

# --- モデルを正規化層付きで構成 ---
norm_layer = NormalizeModule(mean, std)
model = nn.Sequential(norm_layer, mobilenetv2).to(device).eval()
fmodel = fb.PyTorchModel(model, bounds=(0.0, 1.0))
print("✅ Using IMAGENET1K_V1 + meta mean/std + bounds=(0,1)")

# --- ヘルパー関数 ---
def decode_img(img_path):
    img = Image.open(img_path).convert("RGB")
    return transform_base(img).unsqueeze(0)

def to_torch_clipped(x):
    if isinstance(x, np.ndarray):
        t = torch.from_numpy(x)
    elif isinstance(x, torch.Tensor):
        t = x.clone().detach()
    else:
        raise TypeError("Unsupported type")
    if t.dim() == 3:
        t = t.unsqueeze(0)
    return t.float().cpu().clamp(0, 1)

def save_tensor_image(tensor, path):
    t = tensor.squeeze(0).cpu().clamp(0, 1)
    arr = (t.numpy() * 255).astype(np.uint8)
    arr = np.transpose(arr, (1, 2, 0))
    Image.fromarray(arr).save(path)

def save_noise_vis(orig, clipped, path):
    o = orig.squeeze(0).cpu().numpy()
    c = clipped.squeeze(0).cpu().numpy()
    diff = o - c
    mag = np.max(np.abs(diff)) + 1e-8
    vis = (diff / (2 * mag) + 0.5)
    arr = (np.transpose(vis, (1,2,0)) * 255).astype(np.uint8)
    Image.fromarray(arr).save(path)

def extract_number(path: Path):
    m = re.search(r'\d+', path.stem)
    return int(m.group()) if m else float('inf')

# --- 推論 ---
def pred(images, topk=1):
    with torch.no_grad():
        logits = model(images.to(device))
        probs = F.softmax(logits, dim=1)
        top_probs, top_idxs = torch.topk(probs, topk)
        idx = int(top_idxs[0][0].item())
        return idx, imagenet_classes[idx], float(top_probs[0][0].cpu().item())

def lb_to_tensor(idx):
    return torch.tensor([idx], dtype=torch.long, device=device)

# --- FGSM攻撃 ---
def perform_attack(fmodel_local, images, labels, steps=1000, max_eps=1.0):
    attack = fb.attacks.FGSM()
    eps = 0.0
    for _ in range(steps):
        eps += max_eps / steps
        raw, clipped, is_adv = attack(fmodel_local, images, labels, epsilons=[eps])
        if isinstance(clipped, (list, tuple)):
            clipped = clipped[0]
        clipped_t = to_torch_clipped(clipped)
        adv_idx, _, _ = pred(clipped_t.to(device), 1)
        orig_idx, _, _ = pred(images, 1)
        if adv_idx != orig_idx:
            return clipped_t, eps, adv_idx
    return None, eps, orig_idx

# --- 攻撃実行 ---
AE_results = []
steps = 1000
max_eps = 1.0

image_files = sorted([f for f in folder_path.iterdir() if f.suffix.lower() in ['.jpg','.jpeg','.png']], key=extract_number)
for i, img_path in enumerate(tqdm(image_files, desc="FGSM Attack")):
    images = decode_img(str(img_path)).to(device)
    orig_idx, orig_name, orig_conf = pred(images, 1)
    labels = lb_to_tensor(orig_idx)

    clipped, eps, adv_lb = perform_attack(fmodel, images, labels, steps=steps, max_eps=max_eps)

    if clipped is not None:
        adv_idx, adv_name, adv_conf = pred(clipped, 1)
        tensor_path = output_adv_tensor_dir / f"{img_path.stem}.pt"
        torch.save(clipped.cpu(), tensor_path)
        save_tensor_image(clipped, output_adv_dir / f"{img_path.stem}.jpg")
        save_noise_vis(images, clipped, output_noise_dir / f"noise_{i}.jpg")

        AE_results.append({
            "image": img_path.name,
            "Original_label": orig_idx,
            "Original_label_name": orig_name,
            "Original_confidence": orig_conf,
            "Advs_label": adv_idx,
            "Advs_label_name": adv_name,
            "Advs_confidence": adv_conf,
            "Advs_eps": eps,
            "Tensor_path": str(tensor_path)
        })
        print(f"[{i}] ✅ 攻撃成功: Eps={eps:.4f}, {orig_name} → {adv_name}")
    else:
        print(f"[{i}] ❌ 攻撃失敗: {orig_name}")

# --- CSV保存 ---
df = pd.DataFrame(AE_results)
out_csv = output_adv_dir / "FGSM_Results.csv"
df.to_csv(out_csv, index=False, encoding="utf-8-sig")
print(f"\n✅ 攻撃結果を保存しました: {out_csv}")


✅ Using IMAGENET1K_V1 + meta mean/std + bounds=(0,1)


FGSM Attack:   0%|          | 1/1000 [00:00<01:51,  8.93it/s]

[0] ✅ 攻撃成功: Eps=0.0010, marmoset → titi


FGSM Attack:   0%|          | 2/1000 [00:00<02:03,  8.08it/s]

[1] ✅ 攻撃成功: Eps=0.0020, parachute → balloon


...

[994] ✅ 攻撃成功: Eps=0.0030, cabbage butterfly → sulphur butterfly
[995] ✅ 攻撃成功: Eps=0.0030, fountain → guillotine


FGSM Attack: 100%|█████████▉| 997/1000 [03:27<00:00,  6.49it/s]

[996] ✅ 攻撃成功: Eps=0.0030, reflex camera → lens cap
[997] ✅ 攻撃成功: Eps=0.0010, turnstile → bow


FGSM Attack: 100%|██████████| 1000/1000 [03:27<00:00,  4.82it/s]

[998] ✅ 攻撃成功: Eps=0.0030, balloon → conch
[999] ✅ 攻撃成功: Eps=0.0020, sulphur-crested cockatoo → American egret

✅ 攻撃結果を保存しました: C:\Users\sit\wavelet_CGAN\testdata3\data3_FGSM\FGSM_Results.csv


In [10]:
import pandas as pd
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

# === CSV読み込み ===
csv_path = r"C:\Users\sit\wavelet_CGAN\testdata3\data3_FGSM\FGSM_Results.csv"
df = pd.read_csv(csv_path)

# === 必要な列を選択（sample列を除外） ===
columns = [
    "Original_label_name",
    "Original_confidence",
    "Advs_label_name",
    "Advs_confidence",
    "Advs_eps"
]
df_show = df[columns].copy()

# === 先頭100枚のみ抽出 ===
df_show = df_show.head(100)

# === 桁数調整（ε値はそのまま） ===
df_show["Original_confidence"] = df_show["Original_confidence"].round(6)
df_show["Advs_confidence"] = df_show["Advs_confidence"].round(6)

# === 日本語列名 ===
df_show.columns = [
    "入力画像ラベル",
    "confidence",
    "敵対的画像ラベル",
    "confidence(敵対的)",
    "SucEps"
]

# === Word文書作成 ===
doc = Document()
doc.add_heading("攻撃結果表（FGSM）", level=1)
doc.add_paragraph("FGSM攻撃後の入力画像に対する分類結果（先頭100枚）を示す。")

# === 分割設定 ===
interval = 100 # 25行ごとに区切る
col_widths = [
    Inches(1.5),
    Inches(1.0),
    Inches(1.5),
    Inches(1.0),
    Inches(0.8)
]

# === 表を複数に分けて出力 ===
for start in range(0, len(df_show), interval):
    subset = df_show.iloc[start:start + interval]

    # --- 新しい表を作成 ---
    table = doc.add_table(rows=1, cols=len(df_show.columns))
    table.style = 'Table Grid'

    # --- ヘッダー行 ---
    hdr_cells = table.rows[0].cells
    for i, col_name in enumerate(df_show.columns):
        p = hdr_cells[i].paragraphs[0]
        run = p.add_run(col_name)
        run.bold = True
        run.font.name = '游明朝'
        run.font.size = Pt(10.5)
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER

    # --- データ行 ---
    for _, row in subset.iterrows():
        row_cells = table.add_row().cells
        for i, val in enumerate(row):
            if isinstance(val, float) and i in [1, 3]:  # confidence列のみ小数点固定
                text = f"{val:.6f}"
            else:
                text = str(val)
            p = row_cells[i].paragraphs[0]
            run = p.add_run(text)
            run.font.name = '游明朝'
            run.font.size = Pt(10.5)
            p.alignment = WD_ALIGN_PARAGRAPH.CENTER

    # --- 列幅設定 ---
    for row in table.rows:
        for i, width in enumerate(col_widths):
            row.cells[i].width = width

    # --- 表間にスペースを追加 ---
    doc.add_paragraph("")

# === Wordファイル保存 ===
save_path = r"C:\Users\sit\wavelet_CGAN\testdata3\data3_FGSM\Origin_Adv_Table_YuMincho_100.docx"
doc.save(save_path)

print("✅ 先頭100枚のデータを游明朝フォントでWord出力しました：", save_path)


✅ 先頭100枚のデータを游明朝フォントでWord出力しました： C:\Users\sit\wavelet_CGAN\testdata3\data3_FGSM\Origin_Adv_Table_YuMincho_100.docx
